# Basic Chain

In [10]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"))

In [2]:
# ask the LLM a question
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in a few ways:\n\n1. Automated Testing: Langsmith can be used to write automated test scripts that can be run to quickly and efficiently test software applications. This can help identify bugs and issues early in the development process.\n\n2. Test Data Generation: Langsmith can be used to generate test data for various scenarios, helping to ensure that the software is tested under a wide range of conditions.\n\n3. Performance Testing: Langsmith can be used to simulate user interactions and test the performance of software applications under different load conditions. This can help identify potential bottlenecks and optimize performance.\n\n4. Integration Testing: Langsmith can be used to test the integration of different components or services within a software application, ensuring that they work together as expected.\n\nOverall, Langsmith can help streamline the testing process, improve test coverage, and ultimately ensure that soft

In [8]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template("What is pandas DataFrame?{input1} | {input2}")
prompt.format(input1="pandas DataFrame", input2="pandas DataFrame", input3="pandas DataFrame")


'What is pandas DataFrame?pandas DataFrame | pandas DataFrame'

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# create a prompt template with {input} as the user input
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

# you can easily replace placeholder with the user input
print(prompt.format(input="how can langsmith help with testing?"))

# create a simple output parser to parse AIMessge to string
output_parser = StrOutputParser()

# a simple LLM chain using LCEL (langchain expression language)
chain = prompt | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})
type(response), response

System: You are world class technical documentation writer.
Human: how can langsmith help with testing?


(str,
 "Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith provides tools and libraries that enable automated testing of code, which can help ensure the quality and reliability of software products.\n\n2. Test Case Generation: Langsmith can assist in generating test cases for various scenarios, reducing the manual effort required for creating comprehensive test suites.\n\n3. Code Analysis: Langsmith's code analysis capabilities can help identify potential issues and bugs in the codebase, allowing testers to focus on critical areas during testing.\n\n4. Integration Testing: Langsmith can facilitate integration testing by providing utilities to test how different components of a system interact with each other.\n\n5. Performance Testing: Langsmith can assist in performance testing by providing tools to measure and analyze the performance of software applications under different conditions.\n\nOverall, Langsmith can streamline the testing process and impro

# Retrieval Chain

Load documents from web and ingest documents into a vectorstore. Here we use OpenAI's embedding model by default and a light weight local vectorstore, FAISS.

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
loader = WebBaseLoader("https://docs.smith.langchain.com")

embeddings = OpenAIEmbeddings()
docs = loader.load()

In [5]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could pass the documents directly like previous example:

```python
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})
```

But we want to use retrieve the most relevant info from the vectorstore we just created.

In [7]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It also offers tracing and evaluation capabilities to assist with testing LLM applications.


In [8]:
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content="LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\nSkip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction‚ÄãLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start‚ÄãTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps‚ÄãCheck out the following sections to learn more about LangSmith:User Guide: Learn about 